# Exercise
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.
See the end of this notebook for examples of real-world business applications.
## ADDED FUNCTIONALITIES
### MULTISHOT PROMPTING 
### SPECIFIED OUTPUT
### LANGUAGE CONVERSION (SPANISH)



In [1]:
# imports
# Enhanced Web Scraping and Brochure Generator
# Includes multi-shot prompting, structured brochure format, and Spanish translation

import os
import requests
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import time
from urllib.parse import urljoin, urlparse

In [2]:
# Initialize and constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url
        self.title = ""
        self.text = ""
        self.links = []
        
        try:
            # Add headers to avoid blocking
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
            }
            response = requests.get(url, timeout=10, headers=headers)
            response.raise_for_status()
            
            self.body = response.content
            soup = BeautifulSoup(self.body, 'html.parser')
            
            # Get title
            self.title = soup.title.string if soup.title else "No title found"
            self.title = self.title.strip() if self.title else "No title found"
            
            # Get text content
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = ""
            
            # Get links
            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]
            
        except Exception as e:
            print(f"Error scraping {url}: {e}")
            self.title = "Error loading page"
            self.text = f"Could not load content from {url}: {str(e)}"
            self.links = []

In [4]:
   def get_contents(self):
        """Return formatted contents of the webpage"""
        return f"Webpage Title:\n{self.title}\n\nWebpage Contents:\n{self.text}\n\n"


In [5]:
# ENHANCED: Multi-shot prompting for link extraction
link_system_prompt = """You are an expert web analyst. You will be provided with a list of links found on a webpage.
Your task is to identify the most relevant links for creating a comprehensive company brochure.

Look for links that contain information about:
- About/Company information
- Services or Products
- Team/Leadership
- Careers/Jobs
- Company culture
- Case studies or portfolio
- Contact information

Avoid links to:
- Terms of Service, Privacy Policy
- Social media links
- Email addresses
- External websites (unless they're important partnerships)
- Download links for files

Respond in JSON format as shown in the example."""

link_example_prompt = """
Example response format:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about", "relevance": "high"},
        {"type": "services page", "url": "https://company.com/services", "relevance": "high"},
        {"type": "careers page", "url": "https://company.com/careers", "relevance": "medium"},
        {"type": "team page", "url": "https://company.com/team", "relevance": "medium"}
    ]
}
"""


In [6]:
def get_links_user_prompt(website):
    user_prompt = f"""Analyze the links from {website.url}:

Website Title: {website.title}

Links to analyze:
{chr(10).join(website.links[:50])}

Please identify the most relevant links for a company brochure and classify them by type and relevance level."""
    return user_prompt

In [7]:
huggingface = Website("https://huggingface.co")
# huggingface.links

In [8]:
print(get_links_user_prompt(huggingface))

Analyze the links from https://huggingface.co:

Website Title: Hugging Face – The AI community building the future.

Links to analyze:
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/microsoft/VibeVoice-1.5B
/openbmb/MiniCPM-V-4_5
/Qwen/Qwen-Image-Edit
/xai-org/grok-2
/meituan-longcat/LongCat-Flash-Chat
/models
/spaces/enzostvs/deepsite
/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster
/spaces/multimodalart/Qwen-Image-Edit-Fast
/spaces/apple/fastvlm-webgpu
/spaces/multimodalart/nano-banana
/spaces
/datasets/fka/awesome-chatgpt-prompts
/datasets/openai/healthbench
/datasets/syncora/developer-productivity-simulated-behavioral-data
/datasets/facebook/recycling_the_web
/datasets/nvidia/Nemotron-CC-Math-v1
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/allenai
/facebook
/amazon
/google
/Intel
/microsoft
/grammarly
/Writer
/docs/transformers
/docs/diffusers

Plea

In [9]:
def get_links(url):
    """Extract relevant links using multi-shot prompting"""
    website = Website(url)
    
    if not website.links:
        print("No links found on the page")
        return {"links": []}
    
    try:
        # Multi-shot approach: System + Example + User prompt
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": link_example_prompt},
                {"role": "assistant", "content": "I understand. I'll analyze the provided links and return them in the specified JSON format, focusing on company-relevant information and avoiding policy/social media links."},
                {"role": "user", "content": get_links_user_prompt(website)}
            ],
            response_format={"type": "json_object"}
        )
        result = response.choices[0].message.content
        return json.loads(result)
    except Exception as e:
        print(f"Error analyzing links: {e}")
        return {"links": []}


In [10]:
def get_all_details(url):
    """Scrape the main page and relevant linked pages"""
    result = "=== LANDING PAGE ===\n"
    
    # Get main page content
    try:
        landing_page = Website(url)
        result += landing_page.get_contents()
    except Exception as e:
        print(f"Error scraping landing page {url}: {e}")
        result += f"Error loading landing page: {e}\n"
        return result
    
    # Get linked pages
    try:
        links = get_links(url)
        print(f"Found {len(links.get('links', []))} relevant links")
        
        for i, link in enumerate(links.get("links", [])[:5]):  # Limit to 5 most relevant links
            print(f"Scraping {i+1}/5: {link.get('type', 'unknown')} - {link['url']}")
            result += f"\n\n=== {link['type'].upper()} ===\n"
            
            try:
                # Handle relative URLs
                link_url = link["url"]
                if link_url.startswith('/'):
                    link_url = urljoin(url, link_url)
                elif not link_url.startswith('http'):
                    print(f"Skipping invalid URL: {link_url}")
                    continue
                
                link_website = Website(link_url)
                result += link_website.get_contents()
                
            except Exception as e:
                print(f"Error scraping {link['url']}: {e}")
                result += f"Error loading this page: {e}\n"
            
            time.sleep(1)  # Be respectful to the server
            
    except Exception as e:
        print(f"Error processing links: {e}")
    
    return result


In [11]:
# ENHANCED: Structured brochure system prompt with specific format requirements
brochure_system_prompt = """You are a professional marketing content creator specializing in corporate brochures.

Create a comprehensive, well-structured brochure using the provided company information. 

REQUIRED STRUCTURE:
1. **Company Overview** - Brief introduction and mission
2. **Products & Services** - What the company offers
3. **Company Culture & Values** - Work environment and principles
4. **Leadership & Team** - Key people and expertise
5. **Career Opportunities** - Jobs and growth prospects
6. **Why Choose Us** - Competitive advantages
7. **Contact Information** - How to reach them

FORMATTING REQUIREMENTS:
- Use markdown formatting with clear headings
- Include bullet points for easy scanning
- Write in a professional, engaging tone
- Keep each section concise but informative
- Include specific details when available
- Aim for 500-800 words total"""


In [12]:
def get_brochure_user_prompt(company_name, url):
    """Generate the user prompt for brochure creation"""
    user_prompt = f"""Create a professional brochure for: {company_name}
Website: {url}

Use the following information to create a structured brochure following the required format:

{get_all_details(url)}

Please ensure all required sections are included, even if information is limited for some sections."""
    
    # Truncate if too long, but preserve structure
    if len(user_prompt) > 15000:
        user_prompt = user_prompt[:15000] + "\n\n[Content truncated for length - please create brochure with available information]"
    
    return user_prompt

In [13]:
def create_structured_brochure(company_name, url):
    """Create a brochure with multi-shot prompting and structured format"""
    
    # Multi-shot approach for better structure
    example_prompt = """Here's an example of the desired brochure structure:

# TechCorp Solutions

## Company Overview
TechCorp Solutions is a leading provider of innovative software solutions, established in 2010...

## Products & Services  
- Cloud Infrastructure Management
- Custom Software Development
- Digital Transformation Consulting

## Company Culture & Values
We foster an innovative, collaborative environment where...

## Leadership & Team
Our experienced leadership team brings decades of industry expertise...

## Career Opportunities
Join our growing team of talented professionals...

## Why Choose Us
- Industry-leading expertise
- Proven track record
- Customer-focused approach

## Contact Information
- Website: www.techcorp.com
- Email: info@techcorp.com"""

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": example_prompt},
            {"role": "assistant", "content": "I understand the required structure. I'll create a comprehensive brochure following this exact format with all required sections, maintaining professional tone and markdown formatting."},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [14]:
def translate_to_spanish(brochure_content):
    """Translate the entire brochure to Spanish"""
    translation_prompt = f"""Please translate the following company brochure to Spanish. 
    
Maintain the same markdown formatting and structure. Use professional, business-appropriate Spanish.

Brochure to translate:

{brochure_content}"""

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a professional translator specializing in business and marketing content. Translate the provided brochure to Spanish while maintaining the original formatting and professional tone."},
            {"role": "user", "content": translation_prompt}
        ],
    )
    return response.choices[0].message.content


In [15]:
def create_complete_brochure_package(company_name, url, return_content=False):
    """Complete workflow: scrape, create structured brochure, and translate to Spanish"""
    
    print(f"🔍 Starting brochure creation for {company_name}...")
    print(f"📄 Scraping website content from {url}...")
    
    # Step 1: Create English brochure
    english_brochure = create_structured_brochure(company_name, url)
    
    print("✅ English brochure created!")
    print("🇪🇸 Translating to Spanish...")
    
    # Step 2: Translate to Spanish
    spanish_brochure = translate_to_spanish(english_brochure)
    
    print("✅ Spanish translation completed!")
    
    # Display both versions
    print("\n" + "="*60)
    print("ENGLISH BROCHURE")
    print("="*60)
    display(Markdown(english_brochure))
    
    print("\n" + "="*60)
    print("SPANISH BROCHURE (FOLLETO EN ESPAÑOL)")
    print("="*60)
    display(Markdown(spanish_brochure))
    
    # Only return content if explicitly requested
    if return_content:
        return {
            "english": english_brochure,
            "spanish": spanish_brochure
        }
    # Otherwise return None (no output displayed)

In [24]:
def stream_complete_brochure(company_name, url, return_content=False):
    """Streaming version of the complete brochure creation"""
    print(f"🔍 Creating brochure for {company_name}...")
    
    # Create English version with streaming
    print("📄 Creating English brochure...")
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    
    english_response = ""
    display_handle = display(Markdown("# Creating English Brochure...\n"), display_id=True)
    
    for chunk in stream:
        content = chunk.choices[0].delta.content or ''
        english_response += content
        clean_response = english_response.replace("```markdown", "").replace("```", "")
        update_display(Markdown(clean_response), display_id=display_handle.display_id)
    
    print("\n🇪🇸 Creating Spanish translation...")
    
    # Create Spanish version
    spanish_response = translate_to_spanish(english_response)
    
    # Display final results
    print("\n" + "="*60)
    print("FINAL BROCHURES")
    print("="*60)
    display(Markdown("# English Version\n" + english_response))
    display(Markdown("# Versión en Español\n" + spanish_response))

        # Only return content if explicitly requested
    if return_content:
        return {
            "english": english_brochure,
            "spanish": spanish_brochure
        }
    # Otherwise return None (no output displayed)


In [25]:
def test_website_scraping(url):
    """Test the Website class to debug issues"""
    print(f"Testing website scraping for: {url}")
    try:
        site = Website(url)
        print(f"Title: {site.title}")
        print(f"Text length: {len(site.text)} characters")
        print(f"Number of links: {len(site.links)}")
        print(f"First few links: {site.links[:5]}")
        print("✅ Website scraping successful!")
        
        # Test get_contents method
        contents = site.get_contents()
        print(f"Contents method works - length: {len(contents)} characters")
        return site
    except Exception as e:
        print(f"❌ Error in website scraping: {e}")
        import traceback
        traceback.print_exc()
        return None

In [18]:
create_complete_brochure_package("Anthropic", "https://anthropic.com")

🔍 Starting brochure creation for Anthropic...
📄 Scraping website content from https://anthropic.com...
Error scraping landing page https://anthropic.com: 'Website' object has no attribute 'get_contents'
✅ English brochure created!
🇪🇸 Translating to Spanish...
✅ Spanish translation completed!

ENGLISH BROCHURE


# Anthropic

## Company Overview
Founded in 2020, Anthropic is an artificial intelligence research company dedicated to developing reliable and advanced AI systems. Our mission is to advance AI alignment and safety with a focus on building beneficial AI that respects human values and empowers individuals and organizations alike.

## Products & Services  
- **AI Safety Research:** In-depth research aimed at ensuring that artificial intelligence systems are safe and aligned with human intentions.
- **Language Models:** Development of cutting-edge language processing capabilities that redefine how machines understand and generate human language.
- **Consulting & Partnerships:** Collaborating with organizations to implement safe and responsible AI solutions tailored to their unique needs.

## Company Culture & Values
At Anthropic, we believe in fostering a culture of openness, collaboration, and creativity. Our core values include:
- **Integrity:** Upholding ethical standards in all AI developments.
- **Collaboration:** Working together internally and externally to drive innovation.
- **Continuous Learning:** Encouraging personal and career growth through ongoing education and exploration.

## Leadership & Team
Anthropic is led by a group of experts in AI and machine learning, drawing from diverse backgrounds in technology and research. Our leadership team includes:
- **Co-Founders:** Pioneers in AI development with extensive experience at leading tech institutions.
- **Research Scientists:** Specialists in AI alignment and ethical considerations, focused on creating systems that serve humanity.

## Career Opportunities
Anthropic is on the lookout for passionate and talented professionals who are eager to make a difference in the AI landscape. Current opportunities include:
- Research Scientists  
- Software Engineers  
- AI Safety Experts  
Discover your place in a supportive and innovative environment that prioritizes work-life balance and growth potential.

## Why Choose Us
- **Ethical AI Focus:** We prioritize safe and responsible AI, putting integrity at the forefront of our developments.
- **Innovative Research:** Our commitment to research excellence ensures we remain at the cutting edge of AI advancements.
- **Collaborative Environment:** Our team-oriented approach fosters creativity and empowers employees to contribute ideas and initiatives.

## Contact Information
- **Website:** [www.anthropic.com](https://anthropic.com)
- **Email:** contact@anthropic.com
- **Address:** 800 Market St, San Francisco, CA 94102, USA 

---

For those who are passionate about shaping the future of AI, Anthropic offers an exciting opportunity to be part of a critical dialogue and dynamic industry. Join us in our mission to make technology safe and beneficial for everyone.


SPANISH BROCHURE (FOLLETO EN ESPAÑOL)


# Anthropic

## Descripción de la Empresa
Fundada en 2020, Anthropic es una empresa de investigación en inteligencia artificial dedicada a desarrollar sistemas de IA confiables y avanzados. Nuestra misión es avanzar en la alineación y seguridad de la IA con un enfoque en construir una IA beneficiosa que respete los valores humanos y empodere tanto a individuos como a organizaciones.

## Productos y Servicios  
- **Investigación en Seguridad de IA:** Investigación exhaustiva destinada a garantizar que los sistemas de inteligencia artificial sean seguros y estén alineados con las intenciones humanas.
- **Modelos de Lenguaje:** Desarrollo de capacidades de procesamiento del lenguaje de vanguardia que redefinen la manera en que las máquinas entienden y generan el lenguaje humano.
- **Consultoría y Asociaciones:** Colaboración con organizaciones para implementar soluciones de IA seguras y responsables adaptadas a sus necesidades únicas.

## Cultura y Valores de la Empresa
En Anthropic, creemos en fomentar una cultura de apertura, colaboración y creatividad. Nuestros valores fundamentales incluyen:
- **Integridad:** Mantener altos estándares éticos en todos los desarrollos de IA.
- **Colaboración:** Trabajar juntos, tanto interna como externamente, para impulsar la innovación.
- **Aprendizaje Continuo:** Fomentar el crecimiento personal y profesional mediante la educación continua y la exploración.

## Liderazgo y Equipo
Anthropic está liderada por un grupo de expertos en IA y aprendizaje automático, provenientes de diversos antecedentes en tecnología e investigación. Nuestro equipo de liderazgo incluye:
- **Co-Fundadores:** Pioneros en el desarrollo de IA con amplia experiencia en instituciones tecnológicas líderes.
- **Científicos Investigadores:** Especialistas en alineación de IA y consideraciones éticas, enfocados en crear sistemas que sirvan a la humanidad.

## Oportunidades de Carrera
Anthropic está en busca de profesionales apasionados y talentosos que deseen hacer una diferencia en el panorama de la IA. Las oportunidades actuales incluyen:
- Científicos Investigadores  
- Ingenieros de Software  
- Expertos en Seguridad de IA  
Descubre tu lugar en un entorno de apoyo e innovación que prioriza el equilibrio entre vida y trabajo y el potencial de crecimiento.

## ¿Por Qué Elegirnos?
- **Enfoque en IA Ética:** Priorizamos una IA segura y responsable, colocando la integridad en el centro de nuestros desarrollos.
- **Investigación Innovadora:** Nuestro compromiso con la excelencia en la investigación garantiza que permanezcamos a la vanguardia de los avances en IA.
- **Entorno Colaborativo:** Nuestro enfoque orientado al equipo fomenta la creatividad y empodera a los empleados a contribuir con ideas e iniciativas.

## Información de Contacto
- **Sitio web:** [www.anthropic.com](https://anthropic.com)
- **Correo Electrónico:** contact@anthropic.com
- **Dirección:** 800 Market St, San Francisco, CA 94102, EE. UU. 

---

Para aquellos apasionados por dar forma al futuro de la IA, Anthropic ofrece una oportunidad emocionante para ser parte de un diálogo crítico y una industria dinámica. Únete a nosotros en nuestra misión de hacer que la tecnología sea segura y beneficiosa para todos.

In [26]:
stream_complete_brochure("Anthropic", "https://anthropic.com")

🔍 Creating brochure for Anthropic...
📄 Creating English brochure...
Error scraping landing page https://anthropic.com: 'Website' object has no attribute 'get_contents'


# Anthropic Corporate Brochure

## Company Overview
Welcome to Anthropic, where our mission is to advance artificial intelligence that is safe and aligned with human values. Founded by machine learning experts, our team is dedicated to developing AI systems that prioritize ethical considerations while delivering powerful outcomes. As we navigate the complexities of AI, we aim to create technologies that benefit society and foster understanding of AI systems.

## Products & Services
At Anthropic, we specialize in creating advanced AI models and solutions. Our offerings include:

- **Large Language Models:** Cutting-edge natural language processing solutions designed to assist businesses in automating tasks and improving communication.
- **AI Safety Research:** Comprehensive studies on AI behaviors to ensure reliability and mitigate risks associated with advanced systems.
- **Customized AI Solutions:** Tailored applications for various industries, helping clients integrate AI into their existing frameworks seamlessly.

By focusing on safety and alignment, we ensure that our products meet the highest standards of trustworthiness and efficacy.

## Company Culture & Values
At Anthropic, our culture embodies innovation, collaboration, and integrity. We believe in a diverse and inclusive work environment that empowers our team members to thrive:

- **Innovation:** Encouraging creative thinking and exploration of new ideas.
- **Collaboration:** Emphasizing teamwork and open communication to drive success.
- **Integrity:** Upholding ethical standards in our work, foster respect and honesty in our interactions.

We prioritize creating a supportive atmosphere where every employee feels valued and inspired to contribute their best.

## Leadership & Team
Our leadership team comprises industry-leading experts in artificial intelligence and machine learning, bringing years of experience and insight:

- **Co-founders:** A seasoned team of thought leaders who possess deep knowledge in AI's implications and capabilities.
- **Research Scientists and Engineers:** A dedicated group of professionals focused on developing safe AI technologies and exploring innovative approaches.

Our collective expertise positions Anthropic as a leader in the AI field, driving responsible advancements.

## Career Opportunities
At Anthropic, we are always on the lookout for talented individuals who share our vision of creating safe AI. We offer a range of career paths, including:

- **Research Positions:** Work on innovative AI safety research and model training.
- **Engineering Roles:** Collaborate on developing robust AI systems.
- **Support Roles:** Contribute to operational excellence in various capacities.

We provide ample opportunities for growth, learning, and advancement, encouraging our team to lead the AI revolution with us.

## Why Choose Us
Choosing Anthropic means partnering with a company that:

- **Prioritizes Safety in AI:** We are committed to understanding and addressing the potential risks associated with AI, ensuring that our products are built with safety at the forefront.
- **Delivers Ethical Solutions:** We believe in developing AI that aligns with human values, promoting trust and accountability.
- **Invests in Talent:** We nurture our team’s skills and professional growth, fostering an environment where individuals can thrive and make an impact.

Join us in shaping the future of AI, where technology meets humanity.

## Contact Information
Ready to learn more about Anthropic or explore career opportunities? Get in touch with us:

- **Website:** [www.anthropic.com](https://anthropic.com)
- **Email:** info@anthropic.com
- **Phone:** 1-800-ANTHROPIC
- **Address:** 123 AI Valley, San Francisco, CA 94105

Connect with us today and be part of the future of safe and aligned AI.


🇪🇸 Creating Spanish translation...

FINAL BROCHURES


# English Version
# Anthropic Corporate Brochure

## Company Overview
Welcome to Anthropic, where our mission is to advance artificial intelligence that is safe and aligned with human values. Founded by machine learning experts, our team is dedicated to developing AI systems that prioritize ethical considerations while delivering powerful outcomes. As we navigate the complexities of AI, we aim to create technologies that benefit society and foster understanding of AI systems.

## Products & Services
At Anthropic, we specialize in creating advanced AI models and solutions. Our offerings include:

- **Large Language Models:** Cutting-edge natural language processing solutions designed to assist businesses in automating tasks and improving communication.
- **AI Safety Research:** Comprehensive studies on AI behaviors to ensure reliability and mitigate risks associated with advanced systems.
- **Customized AI Solutions:** Tailored applications for various industries, helping clients integrate AI into their existing frameworks seamlessly.

By focusing on safety and alignment, we ensure that our products meet the highest standards of trustworthiness and efficacy.

## Company Culture & Values
At Anthropic, our culture embodies innovation, collaboration, and integrity. We believe in a diverse and inclusive work environment that empowers our team members to thrive:

- **Innovation:** Encouraging creative thinking and exploration of new ideas.
- **Collaboration:** Emphasizing teamwork and open communication to drive success.
- **Integrity:** Upholding ethical standards in our work, foster respect and honesty in our interactions.

We prioritize creating a supportive atmosphere where every employee feels valued and inspired to contribute their best.

## Leadership & Team
Our leadership team comprises industry-leading experts in artificial intelligence and machine learning, bringing years of experience and insight:

- **Co-founders:** A seasoned team of thought leaders who possess deep knowledge in AI's implications and capabilities.
- **Research Scientists and Engineers:** A dedicated group of professionals focused on developing safe AI technologies and exploring innovative approaches.

Our collective expertise positions Anthropic as a leader in the AI field, driving responsible advancements.

## Career Opportunities
At Anthropic, we are always on the lookout for talented individuals who share our vision of creating safe AI. We offer a range of career paths, including:

- **Research Positions:** Work on innovative AI safety research and model training.
- **Engineering Roles:** Collaborate on developing robust AI systems.
- **Support Roles:** Contribute to operational excellence in various capacities.

We provide ample opportunities for growth, learning, and advancement, encouraging our team to lead the AI revolution with us.

## Why Choose Us
Choosing Anthropic means partnering with a company that:

- **Prioritizes Safety in AI:** We are committed to understanding and addressing the potential risks associated with AI, ensuring that our products are built with safety at the forefront.
- **Delivers Ethical Solutions:** We believe in developing AI that aligns with human values, promoting trust and accountability.
- **Invests in Talent:** We nurture our team’s skills and professional growth, fostering an environment where individuals can thrive and make an impact.

Join us in shaping the future of AI, where technology meets humanity.

## Contact Information
Ready to learn more about Anthropic or explore career opportunities? Get in touch with us:

- **Website:** [www.anthropic.com](https://anthropic.com)
- **Email:** info@anthropic.com
- **Phone:** 1-800-ANTHROPIC
- **Address:** 123 AI Valley, San Francisco, CA 94105

Connect with us today and be part of the future of safe and aligned AI.

# Versión en Español
# Folleto Corporativo de Anthropic

## Visión General de la Empresa
Bienvenido a Anthropic, donde nuestra misión es avanzar en la inteligencia artificial que sea segura y esté alineada con los valores humanos. Fundado por expertos en aprendizaje automático, nuestro equipo se dedica a desarrollar sistemas de IA que priorizan las consideraciones éticas al tiempo que ofrecen resultados poderosos. A medida que navegamos por las complejidades de la IA, aspiramos a crear tecnologías que beneficien a la sociedad y fomenten la comprensión de los sistemas de IA.

## Productos y Servicios
En Anthropic, nos especializamos en crear modelos y soluciones de IA avanzados. Nuestras ofertas incluyen:

- **Modelos de Lenguaje Grande:** Soluciones de procesamiento del lenguaje natural de vanguardia diseñadas para ayudar a las empresas a automatizar tareas y mejorar la comunicación.
- **Investigación en Seguridad de IA:** Estudios completos sobre comportamientos de IA para garantizar la fiabilidad y mitigar los riesgos asociados con sistemas avanzados.
- **Soluciones de IA Personalizadas:** Aplicaciones adaptadas para diversas industrias, ayudando a los clientes a integrar la IA en sus marcos existentes de manera fluida.

Al centrarnos en la seguridad y la alineación, garantizamos que nuestros productos cumplan con los más altos estándares de confianza y eficacia.

## Cultura y Valores de la Empresa
En Anthropic, nuestra cultura encarna la innovación, la colaboración y la integridad. Creemos en un entorno laboral diverso e inclusivo que empodera a los miembros de nuestro equipo para prosperar:

- **Innovación:** Fomentar el pensamiento creativo y la exploración de nuevas ideas.
- **Colaboración:** Enfatizar el trabajo en equipo y la comunicación abierta para impulsar el éxito.
- **Integridad:** Mantener estándares éticos en nuestro trabajo, promoviendo el respeto y la honestidad en nuestras interacciones.

Priorizamos la creación de una atmósfera de apoyo donde cada empleado se sienta valorado e inspirado para contribuir con lo mejor de sí mismo.

## Liderazgo y Equipo
Nuestro equipo de liderazgo está compuesto por expertos líderes en la industria de la inteligencia artificial y el aprendizaje automático, que aportan años de experiencia y conocimiento:

- **Co-fundadores:** Un equipo experimentado de líderes de pensamiento que poseen un profundo conocimiento sobre las implicaciones y capacidades de la IA.
- **Científicos e Ingenieros de Investigación:** Un grupo dedicado de profesionales enfocados en desarrollar tecnologías de IA seguras y explorar enfoques innovadores.

Nuestra experiencia colectiva posiciona a Anthropic como un líder en el campo de la IA, impulsando avances responsables.

## Oportunidades de Carrera
En Anthropic, siempre estamos en busca de personas talentosas que compartan nuestra visión de crear IA segura. Ofrecemos una variedad de caminos profesionales, incluyendo:

- **Posiciones de Investigación:** Trabajar en investigaciones innovadoras sobre la seguridad de la IA y el entrenamiento de modelos.
- **Roles de Ingeniería:** Colaborar en el desarrollo de sistemas de IA robustos.
- **Roles de Soporte:** Contribuir a la excelencia operativa en diversas capacidades.

Proporcionamos amplias oportunidades para el crecimiento, el aprendizaje y el avance, animando a nuestro equipo a liderar la revolución de la IA con nosotros.

## Por Qué Elegirnos
Elegir Anthropic significa asociarse con una empresa que:

- **Prioriza la Seguridad en la IA:** Estamos comprometidos a comprender y abordar los riesgos potenciales asociados con la IA, asegurando que nuestros productos se construyan con la seguridad en primer plano.
- **Ofrece Soluciones Éticas:** Creemos en desarrollar IA que esté alineada con los valores humanos, promoviendo la confianza y la responsabilidad.
- **Invierte en Talento:** Fomentamos las habilidades y el crecimiento profesional de nuestro equipo, creando un entorno donde las personas pueden prosperar e impactar.

Únase a nosotros para dar forma al futuro de la IA, donde la tecnología se encuentra con la humanidad.

## Información de Contacto
¿Listo para aprender más sobre Anthropic o explorar oportunidades profesionales? Contáctenos:

- **Sitio Web:** [www.anthropic.com](https://anthropic.com)
- **Correo Electrónico:** info@anthropic.com
- **Teléfono:** 1-800-ANTHROPIC
- **Dirección:** 123 AI Valley, San Francisco, CA 94105

Conéctese con nosotros hoy y sea parte del futuro de una IA segura y alineada.